## We will train the data here

**We will use pytorch nn**

In [84]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader , TensorDataset
import torch.optim as optim
from tqdm import tqdm
import mlflow
import optuna

c:\Users\iiven\anaconda3\envs\AiProjects\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\iiven\anaconda3\envs\AiProjects\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.chdir('..')

In [4]:
os.getcwd()

'c:\\Users\\iiven\\Ai-Cursor\\Machine failure prediction'

In [69]:
df = pd.read_csv('Data/data_final_scaled.csv')


In [70]:
df.head(10)

,footfall,AQ,USS,CS,VOC,RP,IP,Temperature,tempMode,fail
0,-0.001913,-0.243883,0.876321,-2.764959,-0.866163,-0.092400,-1.020413,-2.595761,6,0
1,-0.655739,-0.933358,0.876321,-0.318784,-0.427245,-1.573419,0.840583,-2.595761,1,0
2,-0.655739,0.445591,-1.375365,0.496608,0.889510,1.265202,-1.020413,-2.595761,7,1
3,-0.655739,0.445591,-0.624803,0.496608,0.889510,-0.586073,-1.020413,-2.595761,2,1
4,-0.039274,-0.243883,0.125759,0.496608,0.011673,1.326911,-1.640746,-2.595761,7,0
5,4.388063,0.445591,0.876321,-1.949567,-1.305081,1.203493,-1.020413,-2.595761,2,0
6,0.184895,-1.622832,0.876321,0.496608,-1.305081,2.499385,-0.400081,-2.595761,7,0
7,0.091491,1.135065,-0.624803,-0.318784,0.889510,1.265202,-1.020413,-2.595761,3,1
8,-0.543654,0.445591,-0.624803,-1.134175,-0.427245,1.758875,-1.020413,-2.595761,1,1
9,-0.618377,-0.243883,0.876321,0.496608,-1.305081,1.512038,-1.640746,-2.595761,7,0


In [71]:
train_df, dummy_df = train_test_split(df, test_size=0.2, random_state=42, shuffle= True, stratify= df['fail'])
val_df, test_df = train_test_split(dummy_df, test_size=0.5, random_state=42)


In [72]:
train_df.shape, val_df.shape, test_df.shape


((601, 10), (75, 10), (76, 10))

In [73]:
def df_to_tensors(df):
    X = torch.tensor(df.iloc[:, :-1].values, dtype=torch.float32)
    y = torch.tensor(df.iloc[:, -1].values, dtype=torch.long)
    return X, y

In [74]:
X_train, y_train = df_to_tensors(train_df)
X_val, y_val = df_to_tensors(val_df)
X_test, y_test = df_to_tensors(test_df)

In [75]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape


(torch.Size([601, 9]),
 torch.Size([601]),
 torch.Size([75, 9]),
 torch.Size([75]),
 torch.Size([76, 9]),
 torch.Size([76]))

In [76]:
train_ds = TensorDataset(X_train, y_train)
val_ds = TensorDataset(X_val, y_val)
test_ds = TensorDataset(X_test, y_test)

In [77]:
batch_size = 16
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size)
test_loader = DataLoader(test_ds, batch_size=batch_size)


In [79]:
class ANNModel(nn.Module):
    def __init__(self, input_size, hidden1, hidden2, dropout_rate, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden1),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden1, hidden2),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden2, num_classes)
        )

    def forward(self, x):
        return self.net(x)

In [81]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [86]:
def train_model(model, train_loader, val_loader, optimizer, criterion, epochs):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Training]", leave=False)
        
        for X_batch, y_batch in train_loop:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            train_loop.set_postfix(loss=loss.item())

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X_val_batch, y_val_batch in val_loader:
                X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
                val_outputs = model(X_val_batch)
                _, preds = torch.max(val_outputs, 1)
                correct += (preds == y_val_batch).sum().item()
                total += y_val_batch.size(0)

        val_acc = correct / total
        mlflow.log_metric("val_accuracy", val_acc, step=epoch)
        mlflow.log_metric("train_loss", total_loss, step=epoch)

    return val_acc

In [87]:
def objective(trial):
    input_size = X_train.shape[1]
    num_classes = len(torch.unique(y_train))

    # Hyperparameters to tune
    hidden1 = trial.suggest_int("hidden1", 64, 512)
    hidden2 = trial.suggest_int("hidden2", 64, 512)
    dropout = trial.suggest_float("dropout", 0.2, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    model = ANNModel(input_size, hidden1, hidden2, dropout, num_classes)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    with mlflow.start_run():
        mlflow.log_params({
            "hidden1": hidden1,
            "hidden2": hidden2,
            "dropout": dropout,
            "lr": lr
        })
        val_acc = train_model(model, train_loader, val_loader, optimizer, criterion, epochs=10)
        mlflow.log_metric("final_val_accuracy", val_acc)

    return val_acc

In [88]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)


[I 2025-05-20 17:42:37,092] A new study created in memory with name: no-name-21e742c3-ee77-423d-b850-1b81755e7d66
[I 2025-05-20 17:42:41,973] Trial 0 finished with value: 0.8933333333333333 and parameters: {'hidden1': 179, 'hidden2': 105, 'dropout': 0.35000605896485326, 'lr': 0.009661489045449138}. Best is trial 0 with value: 0.8933333333333333.
[I 2025-05-20 17:42:45,524] Trial 1 finished with value: 0.92 and parameters: {'hidden1': 316, 'hidden2': 315, 'dropout': 0.21012854806855708, 'lr': 0.008067267081653808}. Best is trial 1 with value: 0.92.
[I 2025-05-20 17:42:49,075] Trial 2 finished with value: 0.88 and parameters: {'hidden1': 359, 'hidden2': 179, 'dropout': 0.2731041852309133, 'lr': 0.008437544620241969}. Best is trial 1 with value: 0.92.
[I 2025-05-20 17:42:52,679] Trial 3 finished with value: 0.8666666666666667 and parameters: {'hidden1': 342, 'hidden2': 493, 'dropout': 0.23068842922499136, 'lr': 0.0007626716682150326}. Best is trial 1 with value: 0.92.
[I 2025-05-20 17:42:

In [89]:
study.best_trial

FrozenTrial(number=6, state=TrialState.COMPLETE, values=[0.9333333333333333], datetime_start=datetime.datetime(2025, 5, 20, 17, 42, 59, 302868), datetime_complete=datetime.datetime(2025, 5, 20, 17, 43, 2, 704363), params={'hidden1': 370, 'hidden2': 398, 'dropout': 0.4498854777974932, 'lr': 0.0017328640072318206}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'hidden1': IntDistribution(high=512, log=False, low=64, step=1), 'hidden2': IntDistribution(high=512, log=False, low=64, step=1), 'dropout': FloatDistribution(high=0.5, log=False, low=0.2, step=None), 'lr': FloatDistribution(high=0.01, log=True, low=0.0001, step=None)}, trial_id=6, value=None)